In [2]:
import numpy as np
import numpy.random as random
# import jax.random as random
import jax
import jax.lax as lax
from scipy.stats import rv_discrete

C:\Users\danmo\anaconda3\envs\HMM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\danmo\anaconda3\envs\HMM\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\danmo\anaconda3\envs\HMM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [21]:
import numpy as np

In [67]:
gamma = np.matrix([[1,1,1,1,1],[1,1,1,1,1]])
a = np.matrix([[0.8,0.2],[0.3,0.7]])
b = np.matrix([[0.6,0.2,0.2],[0.3,0.3,0.4]])
beta = np.matrix([[1,2,3,4,5],[1,2,3,4,5]])
o = np.array([0,2,2,1,0])

In [37]:
b[1,2]

0.4

In [26]:
gamma[:, :-1, np.newaxis] * a

ValueError: could not broadcast input array from shape (2,4,1) into shape (2,4)

In [30]:
gamma.shape

(2, 5)

In [29]:
gamma[:,np.newaxis,:].shape

(2, 1, 5)

In [33]:
b[1,[2,2,2]]

matrix([[0.4, 0.4, 0.4]])

In [47]:
np.reshape(o,o.shape[1])

matrix([[0, 2, 2, 1, 0]])

In [42]:
b[:, o[1:]]

matrix([], shape=(2, 0, 5), dtype=float64)

In [40]:
b[np.newaxis, :, o[1:]]

matrix([], shape=(1, 2, 0, 5), dtype=float64)

In [49]:
o

array([0, 2, 2, 1, 0])

In [53]:
a[:,:,np.newaxis].shape

(2, 2, 1)

In [54]:
gamma[:, np.newaxis, :-1].shape

(2, 1, 4)

In [62]:
a[np.newaxis].shape

(1, 2, 2)

In [63]:
np.tensordot?

In [64]:
xsi = (gamma[:, :-1, np.newaxis] * a * b[:, o[1:]].T[np.newaxis, :, :] * beta[:, 1:][:, np.newaxis, :]) / beta[:, :-1][:, np.newaxis, :]

ValueError: could not broadcast input array from shape (2,4,1) into shape (2,4)

In [68]:
a_new = np.repeat(a[:,:, np.newaxis], beta.shape[1]-1, axis=2)
gamma_new = np.repeat(gamma[:,:, np.newaxis], beta.shape[1]-1, axis=2)


ValueError: shape too large to be a matrix.

In [72]:
a[:,:, np.newaxis].shape

(2, 2, 1)

In [75]:
np.repeat( a[:,:, np.newaxis] , 1, axis = 2).shape

(2, 2)

In [69]:
a_new = np.repeat(a[:,:, np.newaxis], beta.shape[1]-1, axis=2)


ValueError: shape too large to be a matrix.

In [79]:
b[:, o[1:]].T[np.newaxis, :, :]

matrix([[[0.2, 0.4],
         [0.2, 0.4],
         [0.2, 0.3],
         [0.6, 0.3]]])

In [87]:
xsi = np.einsum("it,ij,jt,jt ,it -> ijt", gamma[:,:-1], a, b[:, o[1:]] , beta[:, 1:] , (1/ beta[:, :-1]) )

In [89]:
xsi.shape

(2, 2, 4)

In [97]:
import scipy

In [105]:
scipy.stats.norm.pdf([1,2,2],loc = 1, scale = 1)

array([0.39894228, 0.24197072, 0.24197072])

In [110]:
means_ = (0,0)
standard_devs_ = (1,4)
obs_ = (0.5,0.3,0.12)

arr_ = np.array([scipy.stats.norm.pdf(obs_, means_[i], standard_devs_[i]) for i in range(len(means_))])

In [111]:
arr_.shape

(2, 3)

In [91]:
xsi

array([[[0.32      , 0.24      , 0.21333333, 0.6       ],
        [0.16      , 0.12      , 0.08      , 0.075     ]],

       [[0.12      , 0.09      , 0.08      , 0.225     ],
        [0.56      , 0.42      , 0.28      , 0.2625    ]]])

In [96]:
gamma[0,3]

1

In [93]:
xsi[0,1,3]

0.075

In [86]:
xsi = np.einsum("it,ij,jt,jt->it", gamma[:,:-1], a, b[:, o[1:]].T * beta[:, 1:], (1/beta[:, :-1]))


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,4)->(2,4,newaxis) (2,2)->(2,newaxis,2) (4,4)->(4,4) (2,4)->(4,2) 

In [23]:
def xsi(gamma, a, b, beta, o):
    xsi = (gamma[:, :-1, np.newaxis] * a * b[:, o[1:]].T * beta[:, 1:]) / beta[:, :-1]
    return xsi

In [1]:
import jax.numpy as jnp

In [3]:
A = np.array( [[1,2,3] , [4,5,6]])

In [4]:
A

array([[1, 2, 3],
       [4, 5, 6]])

In [5]:
B = A.copy()

In [6]:
B

array([[1, 2, 3],
       [4, 5, 6]])

In [11]:
np.tensordot(A,B, axes = 0).shape

(2, 3, 2, 3)

In [19]:
np.tensordot(A,B, axes=([0,1],[0,1]))

array(91)

In [20]:
A

array([[1, 2, 3],
       [4, 5, 6]])

In [5]:

def sis_estimator(obs, num_bins, iters, num_hidden_states,
                  bin_weight_prior_par=None, latent_prior_par=None,
                  is_mixture=True, output_latents=False):
    if bin_weight_prior_par is None:
        bin_weight_prior_par = jnp.ones(num_bins)
    if latent_prior_par is None:
        latent_prior_par = jnp.ones(num_hidden_states)
    assert num_hidden_states == len(latent_prior_par)
    assert num_bins == len(bin_weight_prior_par)
    if is_mixture:
        num_obs = obs.shape[1]
        obs_dim = obs.shape[0]
        assert obs_dim >= 3
    else:
        obs = jnp.asarray([obs])
        obs = jnp.resize(obs, (min(obs.shape[0], obs.shape[1]), max(obs.shape[0], obs.shape[1])))
        num_obs = obs.shape[1]

    log_evidence_weights = jnp.zeros(iters)

    if output_latents:
        latents_ls = jnp.zeros((iters, num_obs))

    for sis_iter in range(iters):
        if sis_iter % 10 == 0:
            print(f'{sis_iter} Iteration number')
        latents = jnp.zeros(num_obs)
        latent_prob = latent_prior_par / jnp.sum(latent_prior_par)
        latents[0] = rv_discrete(values=(jnp.arange(1, num_hidden_states + 1), latent_prob)).rvs()
        log_evidence_weights[sis_iter] = baseline_log_evidence(obs[:, 0], num_bins, bin_weight_prior_par, is_mixture=is_mixture)

        for data_idx in range(1, num_obs):
            gamma = jnp.zeros(num_hidden_states)
            for state in range(num_hidden_states):
                gamma[state] = gamma_coef(data_idx, state, obs, latents[:data_idx], num_bins, num_hidden_states, bin_weight_prior_par, latent_prior_par, is_mixture)
            if jnp.sum(gamma) == 0:
                print("Error: sum of gamma is equal to zero")
            latent_prob = gamma / jnp.sum(gamma)
            latents[data_idx] = rv_discrete(values=(jnp.arange(1, num_hidden_states + 1), latent_prob)).rvs()
            log_evidence_weights[sis_iter] += jnp.log(jnp.sum(gamma))

        if output_latents:
            latents_ls[sis_iter, :] = latents

    if output_latents:
        return {'evidence': log_evidence_weights, 'latents': latent_ls}
    else:
        return {'evidence': log_evidence_weights }


def gamma_coef(idx, state, obs, latents, num_bins, num_states, bin_weight_prior_par, latent_prior_par, is_mixture=True):
    assert idx <= (len(latents) + 1)
    assert idx >= 2
    if is_mixture:
        obs_upto_idx = jnp.asarray(obs[:, :idx - 1])
        eff_obs = jnp.asarray(obs_upto_idx[:, latents[:idx - 1] == state])
    else:
        obs_upto_idx = jnp.asarray(obs[:, :idx - 1]).T
        eff_obs = jnp.asarray(obs_upto_idx[:, latents[:idx - 1] == state]).T

    log_evid_new = baseline_log_evidence(jnp.column_stack([eff_obs, obs[:, idx]]), num_bins, bin_weight_prior_par, is_mixture)
    log_evid_old = baseline_log_evidence(eff_obs, num_bins, bin_weight_prior_par, is_mixture)
    posterior_latent_weight = post_latent_weight(state, latents[:idx-1], num_states, latent_prior_par, start_state=latents[idx-1], is_mixture=is_mixture)
    evid_ratio = jnp.exp(log_evid_new - log_evid_old)
    return evid_ratio * posterior_latent_weight

def transition_count(latent_states, num_states):
    if len(latent_states) <= 1:
        return jnp.zeros((num_states, num_states))
    sample_size = len(latent_states)
    start_states = latent_states[1:(sample_size-1)]
    end_states = latent_states[2:sample_size]
    transition_count_mat = jnp.zeros((num_states, num_states))
    for i in range(sample_size-1):
        transition_count_mat[start_states[i], end_states[i]] += 1
    return transition_count_mat


def baseline_log_evidence(obs, num_bins, bin_weight_prior_par, is_mixture=True):
    if obs.shape[0] == 0:
        return 0

    if is_mixture:
        obs_dim = obs.shape[0]
        log_evidence = 0
        for dim in range(obs_dim):
            bin_counts = jnp.bincount(obs[dim, :], minlength=num_bins)
            bin_weight_posterior_par = bin_weight_prior_par + bin_counts
            log_evidence += (jnp.sum(lax.lgamma(bin_weight_posterior_par)) - lax.lgamma(jnp.sum(bin_weight_posterior_par))
                             + lax.lgamma(jnp.sum(bin_weight_prior_par)) - jnp.sum(lax.lgamma(bin_weight_prior_par))
                             )
    else:
        bin_counts = jnp.bincount(obs, minlength=num_bins)
        bin_weight_posterior_par = bin_weight_prior_par + bin_counts
        log_evidence = (jnp.sum(lax.lgamma(bin_weight_posterior_par)) - lax.lgamma(jnp.sum(bin_weight_posterior_par))
                        + lax.lgamma(jnp.sum(bin_weight_prior_par)) - jnp.sum(lax.lgamma(bin_weight_prior_par))
                        )
    return log_evidence


def post_latent_weight(state, latents, num_states, latent_prior_par, start_state = None, is_mixture = True):
    if is_mixture:
        state_counts = jnp.bincount(latents, minlength=num_states)
        latent_post_par = latent_prior_par + state_counts
        return latent_post_par[state] / jnp.sum(latent_post_par)
    else:
        # need non-null start state
        if start_state is None:
            raise ValueError("When is_mixture=False, a non-null start_state must be provided")
        transition_counts_from_start_state = transition_count(latent_states = latents, num_states = num_states)[start_state,]
        latent_post_par = latent_prior_par + transition_counts_from_start_state
        return latent_post_par[state] / jnp.sum(latent_post_par)


In [6]:
def simulate_hmm(transition_matrix, means=None, variances=None,
                 emission_probabilities=None, initial_distribution=None,
                 seed=0):
    n_states = transition_matrix.shape[0]
    n_steps = 10
    # Check if shapes of inputs are consistent
    if transition_matrix.shape[0] != n_states:
        raise ValueError(f"Number of rows in transition matrix should be {n_states}")
    if transition_matrix.shape[1] != n_states:
        raise ValueError(f"Number of columns in transition matrix should be {n_states}")
    if means is not None and variances is not None:
        if len(means) != n_states or len(variances) != n_states:
            raise ValueError(f"Number of means and variances should be {n_states}")
    if emission_probabilities is not None:
        if emission_probabilities.shape[0] != n_states:
            raise ValueError(f"Number of rows in observed_categories should be {n_states}")
    # Compute stationary distribution if no initial distribution is provided
    if initial_distribution is None:
        leading_eigenvector = jnp.linalg.eig(transition_matrix.T)[1][:, 0]
        initial_distribution = jnp.abs(leading_eigenvector) / jnp.sum(jnp.abs(leading_eigenvector))
    # Set random seed
    print(f'Computed stat dist')
    random.seed(seed)
    hidden_chain = [random.categorical(key, initial_distribution)]
    observations = []
    if means is not None and variances is not None:
        observations.append(random.normal(subkey, means[hidden_chain[0]], variances[hidden_chain[0]]))
    elif emission_probabilities is not None:
        observations.append(random.categorical(subkey, emission_probabilities[hidden_chain[0]]))
    else:
        raise ValueError("Either means and variances or observed_categories should be provided")
    for i in range(1, n_steps):
        print(f'Iteration {i}')
        random.seed(seed + i )
        hidden_chain.append(random.categorical(subkey, transition_matrix[hidden_chain[i - 1]]))
        random.seed(seed + i * i )
        if means is not None and variances is not None:
            observations.append(random.normal(subkey, means[hidden_chain[i]], variances[hidden_chain[i]]))
        elif emission_probabilities is not None:
            observations.append(random.categorical(subkey, emission_probabilities[hidden_chain[i]]))
    return hidden_chain, observations

In [7]:
trans_mat = jnp.array([[0.7,0.3],[0.2,0.8]])

In [8]:
trans_mat

DeviceArray([[0.7, 0.3],
             [0.2, 0.8]], dtype=float32)

In [9]:
emission_probabilities = jnp.array( [ [0.5,0.4,0.1 ] , [0.1,0.7,0.2] ])

In [10]:
emission_probabilities

DeviceArray([[0.5, 0.4, 0.1],
             [0.1, 0.7, 0.2]], dtype=float32)

In [ ]:
simulated_data = simulate_hmm(transition_matrix=trans_mat,emission_probabilities=emission_probabilities)

In [1]:
simulated_data

NameError: name 'simulated_data' is not defined

In [112]:
a

matrix([[0.8, 0.2],
        [0.3, 0.7]])

In [113]:
b

matrix([[0.6, 0.2, 0.2],
        [0.3, 0.3, 0.4]])

In [116]:
a / np.sum(a, axis = 1)

matrix([[0.8, 0.2],
        [0.3, 0.7]])

In [118]:
np.sum(a,axis=1).shape

(2, 1)